In [ ]:
!pip install pingouin

import pandas as pd
import pingouin as pg
import numpy as np

def calculate_icc(csv_file):
    data = pd.read_csv(csv_file)
    data_long = data.melt(var_name='raters', value_name='ratings', ignore_index=False)
    data_long.reset_index(inplace=True)
    data_long.rename(columns={'index': 'targets'}, inplace=True)
    icc_results = pg.intraclass_corr(data=data_long, targets='targets', raters='raters', ratings='ratings')
    return icc_results, data_long

def main_icc(csv_file):
    icc_results, data_long = calculate_icc(csv_file)
    
    # ICC1とICC2を抽出
    icc1 = icc_results[icc_results['Type'] == 'ICC1']
    icc2 = icc_results[icc_results['Type'] == 'ICC2']
    
    # ICC1の情報を表示
    if not icc1.empty:
        icc1_value = icc1['ICC'].values[0]
        icc1_pval = icc1['pval'].values[0]
        icc1_ci95 = icc1['CI95%'].values[0]
        
        print(f"ICC1 Value: {icc1_value:.4f}")
        print(f"ICC1 p-value: {icc1_pval:.4f}")
        print(f"ICC1 95% Confidence Interval (CI95%): {icc1_ci95}")
        
        # SEMの計算（ICC1を使用）
        total_variance = np.var(data_long['ratings'], ddof=1)
        sem1 = total_variance * np.sqrt(1 - icc1_value)
        print(f"SEM1 (from ICC1): {sem1:.4f}")
        
        # 平均差のSDを使用したSEMの計算
        ratings_diff = data_long.groupby('targets')['ratings'].diff().dropna()
        sd_diff = np.std(ratings_diff, ddof=1)
        sem_diff1 = sd_diff / np.sqrt(2)
        print(f"SEM (using SD of mean differences for ICC1): {sem_diff1:.4f}")
        
        # MDC95の計算
        mdc95_1 = sem1 * 1.96 * np.sqrt(2)
        mdc95_diff1 = sem_diff1 * 1.96 * np.sqrt(2)
        print(f"MDC95 (from ICC1): {mdc95_1:.4f}")
        print(f"MDC95 (using SD of mean differences for ICC1): {mdc95_diff1:.4f}")
    else:
        print("ICC1 is not found in the results.")
    
    # ICC2の情報を表示
    if not icc2.empty:
        icc2_value = icc2['ICC'].values[0]
        icc2_pval = icc2['pval'].values[0]
        icc2_ci95 = icc2['CI95%'].values[0]
        
        print(f"\nICC2 Value: {icc2_value:.4f}")
        print(f"ICC2 p-value: {icc2_pval:.4f}")
        print(f"ICC2 95% Confidence Interval (CI95%): {icc2_ci95}")
        
        # SEMの計算（ICC2を使用）
        sem2 = total_variance * np.sqrt(1 - icc2_value)
        print(f"SEM2 (from ICC2): {sem2:.4f}")
        
        # 平均差のSDを使用したSEMの計算
        sem_diff2 = sd_diff / np.sqrt(2)
        print(f"SEM (using SD of mean differences for ICC2): {sem_diff2:.4f}")
        
        # MDC95の計算
        mdc95_2 = sem2 * 1.96 * np.sqrt(2)
        mdc95_diff2 = sem_diff2 * 1.96 * np.sqrt(2)
        print(f"MDC95 (from ICC2): {mdc95_2:.4f}")
        print(f"MDC95 (using SD of mean differences for ICC2): {mdc95_diff2:.4f}")
    else:
        print("ICC2 is not found in the results.")

# 使用例
main_icc('deta.csv')
